In [ ]:
import random
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

# ANSI color codes for better readability
GREEN = "\033[1;32m"
YELLOW = "\033[1;33m"
GRAY = "\033[1;30m"
RESET = "\033[0m"

def load_words(filename):
    with open(filename, 'r') as file:
        words = [line.strip().lower() for line in file]
    return words

class WordleAI:
    def __init__(self, words, lr=0.001, gamma=0.95, epsilon=0.1):
        self.words = words
        self.word_to_idx = {word: idx for idx, word in enumerate(words)}
        self.idx_to_word = {idx: word for word, idx in self.word_to_idx.items()}
        self.state_size = len(words)  # state is a one-hot vector of word choices
        self.action_size = len(words)
        self.gamma = gamma  # discount rate
        self.learning_rate = lr
        self.epsilon = epsilon  # exploration rate
        self.model = self.build_model()
        
    def build_model(self):
        inputs = Input(shape=(self.state_size,))
        x = Dense(128, activation='relu')(inputs)
        x = Dense(64, activation='relu')(x)
        outputs = Dense(self.action_size, activation='linear')(x)
        model = Model(inputs=inputs, outputs=outputs)
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def get_state(self, possible_words):
        state = np.zeros(self.state_size)
        for word in possible_words:
            idx = self.word_to_idx[word]
            state[idx] = 1
        return state.reshape(1, -1)

    def choose_action(self, state):
        if np.random.rand() < self.epsilon:
            return random.randint(0, self.action_size - 1)  # Random action
        q_values = self.model.predict(state)
        return np.argmax(q_values[0])

    def train(self, secret_word, episodes=5000):
        for episode in range(episodes):
            possible_words = self.words.copy()
            state = self.get_state(possible_words)
            
            for turn in range(6):  # limit to 6 guesses
                action_idx = self.choose_action(state)
                guess = self.idx_to_word[action_idx]
                
                reward, feedback = self.get_feedback_and_reward(guess, secret_word)
                if guess == secret_word:
                    break

                new_possible_words = self.update_possible_words(possible_words, feedback)
                next_state = self.get_state(new_possible_words)
                
                q_values = self.model.predict(state)
                next_q_values = self.model.predict(next_state)
                target = reward + self.gamma * np.amax(next_q_values[0])
                q_values[0][action_idx] = target

                self.model.fit(state, q_values, epochs=1, verbose=0)
                state = next_state
                possible_words = new_possible_words

    def get_feedback_and_reward(self, guess, secret_word):
        feedback = []
        reward = -1  # Small penalty for each guess to encourage efficiency
        for i, char in enumerate(guess):
            if char == secret_word[i]:
                feedback.append((char, i, "green"))
                reward += 3
            elif char in secret_word:
                feedback.append((char, i, "yellow"))
                reward += 1
            else:
                feedback.append((char, i, "gray"))
                reward -= 1
        if guess == secret_word:
            reward += 10
        return reward, feedback

    def update_possible_words(self, possible_words, feedback):
        new_possible_words = []
        for word in possible_words:
            match = True
            for char, pos, color in feedback:
                if color == "green" and word[pos] != char:
                    match = False
                elif color == "yellow" and (word[pos] == char or char not in word):
                    match = False
                elif color == "gray" and char in word:
                    match = False
            if match:
                new_possible_words.append(word)
        return new_possible_words

    def play(self, secret_word):
        possible_words = self.words.copy()
        state = self.get_state(possible_words)
        for turn in range(6):
            action_idx = self.choose_action(state)
            guess = self.idx_to_word[action_idx]
            if guess == secret_word:
                return turn + 1  # Return number of attempts taken

            reward, feedback = self.get_feedback_and_reward(guess, secret_word)
            possible_words = self.update_possible_words(possible_words, feedback)
            state = self.get_state(possible_words)
        return 6  # Max attempts if word not guessed

if __name__ == "__main__":
    words = load_words("words.txt")
    ai = WordleAI(words)
    secret_word = random.choice(words)
    print(f"Training with secret word: {secret_word} (hidden for testing)\n")
    ai.train(secret_word, episodes=5000)


Training with secret word: minim (hidden for testing)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━

In [ ]:
attempts_distribution = [0] * 6
total_attempts = 0
num_tests = 10000

for _ in range(num_tests):
    test_word = random.choice(words)
    attempts = ai.play(test_word)
    attempts_distribution[attempts - 1] += 1
    total_attempts += attempts

average_attempts = total_attempts / num_tests
print("\nStatistics after testing:")
for i, count in enumerate(attempts_distribution):
    print(f"{i + 1} attempts: {count} times")
print(f"Average attempts to guess the word: {average_attempts:.2f}")